In [21]:
import pandas as pd
import sqlite3
from sklearn.preprocessing import LabelEncoder

# import files and creation of database in sql

In [93]:
conn = sqlite3.connect('ca2.db')

In [117]:
labour_pool = pd.read_csv('lfsa_epgn62$defaultview_linear.csv')
labour_pool.to_sql('labour_pool', conn, if_exists='replace', index=False)

17369

In [118]:
labour_pool.head()

,DATAFLOW,LAST UPDATE,freq,unit,sex,age,nace_r2,worktime,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:LFSA_EPGN62$DEFAULTVIEW(1.0),27/04/23 02:29:00,A,THS_PER,F,Y15-64,A,FT,AT,2013,45.9,NaN
1,ESTAT:LFSA_EPGN62$DEFAULTVIEW(1.0),27/04/23 02:29:00,A,THS_PER,F,Y15-64,A,FT,AT,2014,50.6,NaN
2,ESTAT:LFSA_EPGN62$DEFAULTVIEW(1.0),27/04/23 02:29:00,A,THS_PER,F,Y15-64,A,FT,AT,2015,46.9,NaN
3,ESTAT:LFSA_EPGN62$DEFAULTVIEW(1.0),27/04/23 02:29:00,A,THS_PER,F,Y15-64,A,FT,AT,2016,41.5,NaN
4,ESTAT:LFSA_EPGN62$DEFAULTVIEW(1.0),27/04/23 02:29:00,A,THS_PER,F,Y15-64,A,FT,AT,2017,40.5,NaN


In [96]:
Precarious_employment = pd.read_csv('lfsa_qoe_4ax1r2_linear.csv')
Precarious_employment.to_sql('Precarious_employment', conn, if_exists='replace', index=False)

34868

In [97]:
National_accounts_employment = pd.read_csv('nama_10_a64_e_linear.csv')
National_accounts_employment.to_sql('National_accounts_employment', conn, if_exists='replace', index=False)

1096326

In [98]:
labour_productivity = pd.read_csv('tin00152_linear.csv')
labour_productivity.to_sql('labour_productivity', conn, if_exists='replace', index=False)

4829

In [99]:
cost = pd.read_csv('sts_copi_a_linear.csv')
cost.to_sql('cost', conn, if_exists='replace', index=False)

3327

In [100]:
investment = pd.read_csv('tipsna50_linear.csv')
investment.to_sql('investment', conn, if_exists='replace', index=False)

756

In [101]:
production = pd.read_csv('sts_copr_a_linear.csv')
production.to_sql('production', conn, if_exists='replace', index=False)


13428

In [119]:

housing_price = pd.read_csv('prc_hpi_a$defaultview_linear.csv')
housing_price.to_sql('housing_price', conn, if_exists='replace', index=False)

2800

In [162]:

population = pd.read_csv('nama_10_pe_linear.csv')
population.to_sql('population', conn, if_exists='replace', index=False)

15304

# query tables

In [185]:
query = "SELECT geo,TIME_PERIOD,OBS_VALUE as'production' FROM  production WHERE nace_r2 = 'F' AND s_adj = 'CA' GROUP BY geo,TIME_PERIOD"


production = pd.read_sql_query(query, conn)

In [186]:
query = "SELECT geo,TIME_PERIOD,OBS_VALUE as'labour_productivity' FROM  labour_productivity WHERE nace_r2 = 'F'  GROUP BY geo,TIME_PERIOD"


labour_productivity = pd.read_sql_query(query, conn)

In [187]:
query = "SELECT geo,TIME_PERIOD,OBS_VALUE as 'Precarious_employment' FROM  Precarious_employment WHERE nace_r2 = 'B-F' AND sex = 'T' GROUP BY geo,TIME_PERIOD"


Precarious_employment = pd.read_sql_query(query, conn)

In [188]:
query = "SELECT geo,TIME_PERIOD,OBS_VALUE as 'labour_pool' FROM labour_pool WHERE nace_r2 = 'F' AND sex != 'T' GROUP BY geo,TIME_PERIOD"


labour_pool = pd.read_sql_query(query, conn)

In [189]:
query = "SELECT geo,TIME_PERIOD,OBS_VALUE as 'investment' FROM investment"

investment = pd.read_sql_query(query, conn)



In [190]:
query = "SELECT geo,TIME_PERIOD,OBS_VALUE as 'cost' FROM cost WHERE unit = 'I15' AND indic_bt ='CSTO' GROUP BY geo,TIME_PERIOD,indic_bt"

cost = pd.read_sql_query(query, conn)

In [191]:
query = "SELECT unit,na_item,geo,TIME_PERIOD,OBS_VALUE FROM  National_accounts_employment WHERE nace_r2 = 'F' AND unit IN ('THS_HW', 'THS_JOB','THS_PER') "


National_accounts_employment = pd.read_sql_query(query, conn)

In [192]:

query = "SELECT purchase,geo,TIME_PERIOD,OBS_VALUE as'housing_price' FROM  housing_price WHERE unit ='I15_A_AVG' "


housing_price = pd.read_sql_query(query, conn)

In [193]:

query = "SELECT geo,TIME_PERIOD,OBS_VALUE,na_item  FROM  population WHERE na_item IN('POP_NC','EMP_NC') AND unit = 'THS_PER'  GROUP BY geo,TIME_PERIOD,na_item "


population = pd.read_sql_query(query, conn)

# data transformation

In [195]:
housing_price = pd.pivot_table(housing_price, values='housing_price', index=['geo','TIME_PERIOD'], columns=['purchase'])

In [196]:
population = pd.pivot_table(population, values='OBS_VALUE', index=['geo','TIME_PERIOD'], columns=['na_item'])

In [197]:
housing_price.reset_index(inplace=True)

In [198]:
population.reset_index(inplace=True)

In [199]:
National_accounts_employment['OBS_VALUE'] = pd.to_numeric(National_accounts_employment['OBS_VALUE'], errors='coerce')

In [200]:
National_accounts_employment = pd.pivot_table(National_accounts_employment, values='OBS_VALUE', index=['geo','TIME_PERIOD'], columns=['na_item', 'unit'])

In [201]:
nuovi_nomi_colonne = ['Total Employment, Hours worked', 'Total Employment, Jobs', 'Total Employment, Persons',
                      'Employees, Hours worked', 'Employees, Jobs', 'Employees, Persons',
                      'Self Employed, Hours worked', 'Self Employed, Jobs', 'Self Employed, Persons'
                     ]

National_accounts_employment.columns = nuovi_nomi_colonne

In [202]:
 National_accounts_employment= National_accounts_employment.reset_index()

In [203]:
National_accounts_employment= National_accounts_employment[['geo','TIME_PERIOD','Total Employment, Persons','Employees, Persons','Self Employed, Persons']]

In [211]:


final = pd.merge(labour_productivity, Precarious_employment, on=['geo', 'TIME_PERIOD'])
final = pd.merge(final, labour_pool, on=['geo', 'TIME_PERIOD'])
final = pd.merge(final, production, on=['geo', 'TIME_PERIOD'])
final = pd.merge(final, investment, on=['geo', 'TIME_PERIOD'])
final = pd.merge(final, cost, on=['geo', 'TIME_PERIOD'])
final = pd.merge(final, housing_price, on=['geo', 'TIME_PERIOD'])
final = pd.merge(final, population, on=['geo', 'TIME_PERIOD'])

In [212]:
time = [2014, 2015, 2016, 2017, 2018, 2019, 2020,2021,2022]

In [213]:
final=final[final['TIME_PERIOD'].isin(time)]

In [215]:
countries = ['ES','IE','DE','IT','FR']

In [216]:
final=final[final['geo'].isin(countries)]

In [209]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 40 to 118
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   geo                    35 non-null     object 
 1   TIME_PERIOD            35 non-null     int64  
 2   labour_productivity    35 non-null     float64
 3   Precarious_employment  34 non-null     float64
 4   labour_pool            35 non-null     float64
 5   production             34 non-null     float64
 6   investment             35 non-null     float64
 7   cost                   35 non-null     float64
 8   DW_EXST                35 non-null     float64
 9   DW_NEW                 35 non-null     float64
 10  TOTAL                  35 non-null     float64
 11  EMP_NC                 35 non-null     float64
 12  POP_NC                 35 non-null     float64
dtypes: float64(11), int64(1), object(1)
memory usage: 3.8+ KB


In [217]:
correlations = final.corr()
correlations

,TIME_PERIOD,labour_productivity,Precarious_employment,labour_pool,production,investment,cost,DW_EXST,DW_NEW,TOTAL,EMP_NC,POP_NC
TIME_PERIOD,1.000000,0.179543,0.054749,0.056978,0.390889,0.161537,0.738788,0.689947,0.787215,0.710654,0.034084,0.010843
labour_productivity,0.179543,1.000000,-0.072536,-0.018664,0.551644,-0.190318,0.329544,0.375926,0.221253,0.356555,-0.247715,-0.348181
Precarious_employment,0.054749,-0.072536,1.000000,0.018661,-0.480285,0.322695,-0.158050,-0.299199,-0.185415,-0.285280,-0.028867,0.184545
labour_pool,0.056978,-0.018664,0.018661,1.000000,-0.018890,0.904342,0.346622,0.086487,0.053745,0.078034,0.933098,0.874650
production,0.390889,0.551644,-0.480285,-0.018890,1.000000,-0.237655,0.445838,0.605750,0.527580,0.601463,-0.196708,-0.381536
investment,0.161537,-0.190318,0.322695,0.904342,-0.237655,1.000000,0.297061,0.072272,0.124047,0.075011,0.870055,0.897050
cost,0.738788,0.329544,-0.158050,0.346622,0.445838,0.297061,1.000000,0.789181,0.708521,0.782656,0.277318,0.182788
DW_EXST,0.689947,0.375926,-0.299199,0.086487,0.605750,0.072272,0.789181,1.000000,0.952170,0.998946,-0.051873,-0.168831
DW_NEW,0.787215,0.221253,-0.185415,0.053745,0.527580,0.124047,0.708521,0.952170,1.000000,0.964737,-0.063795,-0.144989
TOTAL,0.710654,0.356555,-0.285280,0.078034,0.601463,0.075011,0.782656,0.998946,0.964737,1.000000,-0.057701,-0.170296
